##ALUNOS: Lucas Gabriel Nunes Geremias, Joel Sepulveda Martins, Larissa Raimee Gomes e Diego Henrick Candido
##TURMA: U
##NOTURNO

In [ ]:
# Cria o TXT com as formulas para serem convertidas
expr1 = "∀x(P(x) -> Q(x)) ∧ ∃x(P(x) ∧ R(x))"
expr2 = "∃x(P(x) ∧ ∀y(Q(y) -> R(x,y)))"
expr3 = "∀x∀y(P(x,y) <-> P(y,x))"
expr4 = "∃x∀y(P(x,y) ∧ ~Q(y))"
expr5 = "∀x(P(x) -> ∃y(Q(y) ∧ R(x,y)))"

expressions = [expr1, expr2, expr3, expr4, expr5]

# Abrindo um arquivo txt para escrita
file = open("Filebase_v1.txt", "w")

# Escrevendo cada expressão no arquivo
for expr in expressions:
  file.write(expr + "\n")

# Fechando o arquivo
file.close()

In [ ]:
##ALUNOS: Lucas Gabriel Nunes Geremias, Joel Sepulveda Martins
##TURMA: U
##NOTURNO
from IPython.display import display, Latex
def convert_to_latex(formula):
  latex_dict = {
      '∧': "\\wedge ",
      '∨':"\\vee ",
      '~': "\\neg ",
      '->': "\\rightarrow ",
      '<->': "\\leftrightarrow ",
      '∀': "\\forall ",
      '∃': "\\exists ",
      '≡': "\\equiv "
  }
  formula = formula.replace('∧', latex_dict['∧'])
  formula = formula.replace('∨', latex_dict['∨'])
  formula = formula.replace('~', latex_dict['~'])
  formula = formula.replace('->', latex_dict['->'])
  formula = formula.replace('<->', latex_dict['<->'])
  formula = formula.replace('∀', latex_dict['∀'])
  formula = formula.replace('∃', latex_dict['∃'])
  formula = formula.replace('≡', latex_dict['≡'])

  return formula
def delete_quantificator(f):
  have_quantificator = False
  if('∀' in f):
    position = f.find('∀')
    if(f[position + 2] == '∀' or f[position + 2] == '∃'):
      f = f[position + 4: len(f) +1]
      have_quantificator = True
    else:
      f = f[position + 2: len(f) +1]
      have_quantificator = True
  elif('∃' in f):
    position = f.find('∃')
    if(f[position + 2] == '∀' or f[position + 2] == '∃'):
      f = f[position + 4: len(f) + 1]
      have_quantificator = True
    else:
      f = f[position + 2: len(f) + 1]
      have_quantificator = True
  return f, have_quantificator

def transform_dis(formula):
  have_transformation = False
  formula_in_process = formula.split()
  if('<->' in formula_in_process):
    position = formula_in_process.index('<->')
    element_1 = formula_in_process[position - 1]
    element_1_without_quantificator, have_quant = delete_quantificator(element_1)
    if(have_quant == False):
      element_1 = element_1.replace('(', '', 1)
    element_2_base = formula_in_process[position + 1]
    element_2 = element_2_base.replace(')', ' ')
    if(element_2_base[len(element_2_base)- 2] == ')'):
      element_2 = element_2.replace(' ', ')', 1)
    element_2 = element_2.replace(' ', '')

    if(have_quant == False):
      formula_in_process[position - 1] = '(' + element_1 + ' -> ' + element_2 + ')'
      formula_in_process[position + 1] = '(' + element_2 + ' -> ' + element_1 + ')'
    else:
      formula_in_process[position - 1] = element_1 + ' -> ' + element_2 + ')'
      element_1_without_quantificator = element_1_without_quantificator.replace('(', '', 1)
      formula_in_process[position + 1] = '(' + element_2 + ' -> ' + element_1_without_quantificator + ')'
    formula_in_process[position] = '∧'
    have_transformation = True
  elif('->' in formula_in_process):
    position = formula_in_process.index('->')
    formula_in_process[position] = 'V'
    element_base = formula_in_process[position - 1]
    element = element_base.replace('(', '(~', 1)
    formula_in_process[position - 1] = element
    have_transformation = True
    #Verifica se tem '~(' nas strings de formula_in_process
  elif(any('~(' in s for s in formula_in_process)):
    #Retorna a posição de '~(' dentro de formula_in_process
    position = next((i for i, s in enumerate(formula_in_process) if '~(' in s), None)
    if(formula_in_process[position + 1] == 'V'):
      element_base = formula_in_process[position]
      element = element_base.replace('~(', '(~')
      element_base_2 = formula_in_process[position + 2]
      element_2 = element_base_2.replace(element_base_2, '~' + element_base_2)
      formula_in_process[position] = element
      formula_in_process[position + 1] = '∧'
      formula_in_process[position + 2] = element_2
      have_transformation = True
    elif(formula_in_process[position + 1] == '∧'):
      element_base = formula_in_process[position]
      element = element_base.replace('~(', '(~')
      element_base_2 = formula_in_process[position + 2]
      element_2 = element_base_2.replace(element_base_2, '~' + element_base_2)
      formula_in_process[position] = element
      formula_in_process[position + 1] = 'V'
      formula_in_process[position + 2] = element_2
      have_transformation = True
  elif(any('~~' in s for s in formula_in_process)):
    position = next((i for i, s in enumerate(formula_in_process) if '~~' in s), None)
    element_base = formula_in_process[position]
    element = element_base.replace('~~', '')
    formula_in_process[position] = element
    have_transformation = True
  elif(any('V' in s for s in formula_in_process)):
    position = next((i for i, s in enumerate(formula_in_process) if 'V' in s), None)
    next_ = formula_in_process[position + 1]
    before_ = formula_in_process[position - 1]
    #Conta os parenteses
    count = 0
    for i in next_:
      if(i == ')'):
        count+=1
    if('(' == next_[0] and formula_in_process[position + 2] == '∧' and count < 2):
      element_1 = formula_in_process[position - 1]

      element_2 = formula_in_process[position + 1]
      element_2 = element_2.replace('(', '', 1)

      element_3 = formula_in_process[position + 3]
      element_3 = element_3.replace(')', ' ')
      element_3 = element_3.replace(' ', ')', 1)
      element_3 = element_3.replace(' ', '')

      formula_in_process[position - 1 ] = "(" + element_1 + " V " + element_2 + ")"
      formula_in_process[position] = '∧'
      formula_in_process[position + 1] = "(" + element_1 + " V " + element_3 + ")"
      del formula_in_process[position + 2]
      del formula_in_process[position + 2]
      have_transformation = True
      # and count < 2
    elif(')' == before_[len(before_) - 1] and formula_in_process[position - 2] == '∧' ):
      element_1 = formula_in_process[position + 1]

      element_2_base = formula_in_process[position - 1]
      element_2 = element_2_base.replace(')', ' ')
      if(element_2_base[len(element_2_base) - 2] == ')'):
        element_2 = element_2.replace(' ', ')', 1)
      element_2 = element_2.replace(' ', '')

      element_3 = formula_in_process[position - 3]
      element_3 = element_3.replace('(', '', 1)

      formula_in_process[position - 1 ] = "(" + element_3 + " V " + element_1 + ")"
      formula_in_process[position] = '∧'
      formula_in_process[position + 1] = "(" + element_2 + " V " + element_1 + ")"
      del formula_in_process[position - 2]
      del formula_in_process[position - 3]
      have_transformation = True
  if(have_transformation == False):
    if(any('∧' in s for s in formula_in_process)):
      position = next((i for i, s in enumerate(formula_in_process) if '∧' in s), None)
      next_ = formula_in_process[position + 1]
      before_ = formula_in_process[position - 3]
      before_without_quantificator, s = delete_quantificator(before_)
      next_without_quantificator, s = delete_quantificator(next_)
      formula_in_process[position - 3] = before_.replace("(", "~(", 1)
      count = 0
      for i in next_:
        if(i == '('):
          count+=1
      if(count > 1):
        formula_in_process[position + 1] = next_.replace("(", "~(", 1)
      else:
        formula_in_process[position + 1] = next_.replace(next_,'~' + next_)
      formula_in_process[position] = 'V'
  formula_final = ' '.join(formula_in_process)
  return formula_final, have_transformation

#------------------------------------ TRANSFORMA EM CONJUNTIVA ------------------------------------
def transform_conj(formula):
  have_transformation = False
  formula_in_process = formula.split()
  if('<->' in formula_in_process):
    position = formula_in_process.index('<->')
    element_1 = formula_in_process[position - 1]
    element_1_without_quantificator, have_quant = delete_quantificator(element_1)
    if(have_quant == False):
      element_1 = element_1.replace('(', '', 1)
    element_2_base = formula_in_process[position + 1]
    element_2 = element_2_base.replace(')', ' ')
    if(element_2_base[len(element_2_base)- 2] == ')'):
      element_2 = element_2.replace(' ', ')', 1)
    element_2 = element_2.replace(' ', '')

    if(have_quant == False):
      formula_in_process[position - 1] = '(' + element_1 + ' -> ' + element_2 + ')'
      formula_in_process[position + 1] = '(' + element_2 + ' -> ' + element_1 + ')'
    else:
      formula_in_process[position - 1] = element_1 + ' -> ' + element_2 + ')'
      element_1_without_quantificator = element_1_without_quantificator.replace('(', '', 1)
      formula_in_process[position + 1] = '(' + element_2 + ' -> ' + element_1_without_quantificator + ')'
    formula_in_process[position] = '∧'
    have_transformation = True
  elif('->' in formula_in_process):
    position = formula_in_process.index('->')
    formula_in_process[position] = 'V'
    element_base = formula_in_process[position - 1]
    element = element_base.replace('(', '(~', 1)
    formula_in_process[position - 1] = element
    have_transformation = True
    #Verifica se tem '~(' nas strings de formula_in_process
  elif(any('~(' in s for s in formula_in_process)):
    #Retorna a posição de '~(' dentro de formula_in_process
    position = next((i for i, s in enumerate(formula_in_process) if '~(' in s), None)
    if(formula_in_process[position + 1] == 'V'):
      element_base = formula_in_process[position]
      element = element_base.replace('~(', '(~')
      element_base_2 = formula_in_process[position + 2]
      element_2 = element_base_2.replace(element_base_2, '~' + element_base_2)
      formula_in_process[position] = element
      formula_in_process[position + 1] = '∧'
      formula_in_process[position + 2] = element_2
      have_transformation = True
    elif(formula_in_process[position + 1] == '∧'):
      element_base = formula_in_process[position]
      element = element_base.replace('~(', '(~')
      element_base_2 = formula_in_process[position + 2]
      element_2 = element_base_2.replace(element_base_2, '~' + element_base_2)
      formula_in_process[position] = element
      formula_in_process[position + 1] = 'V'
      formula_in_process[position + 2] = element_2
      have_transformation = True
  elif(any('~~' in s for s in formula_in_process)):
    position = next((i for i, s in enumerate(formula_in_process) if '~~' in s), None)
    element_base = formula_in_process[position]
    element = element_base.replace('~~', '')
    formula_in_process[position] = element
    have_transformation = True
  elif(any('V' in s for s in formula_in_process)):
    position = next((i for i, s in enumerate(formula_in_process) if 'V' in s), None)
    next_ = formula_in_process[position + 1]
    before_ = formula_in_process[position - 1]
    #Conta os parenteses
    count = 0
    for i in next_:
      if(i == ')'):
        count+=1
    if('(' == next_[0] and formula_in_process[position + 2] == '∧' and count < 2):
      element_1 = formula_in_process[position - 1]

      element_2 = formula_in_process[position + 1]
      element_2 = element_2.replace('(', '', 1)

      element_3 = formula_in_process[position + 3]
      element_3 = element_3.replace(')', ' ')
      element_3 = element_3.replace(' ', ')', 1)
      element_3 = element_3.replace(' ', '')

      formula_in_process[position - 1 ] = "(" + element_1 + " V " + element_2 + ")"
      formula_in_process[position] = '∧'
      formula_in_process[position + 1] = "(" + element_1 + " V " + element_3 + ")"
      del formula_in_process[position + 2]
      del formula_in_process[position + 2]
      have_transformation = True
      # and count < 2
    elif(')' == before_[len(before_) - 1] and formula_in_process[position - 2] == '∧' ):
      element_1 = formula_in_process[position + 1]

      element_2_base = formula_in_process[position - 1]
      element_2 = element_2_base.replace(')', ' ')
      if(element_2_base[len(element_2_base) - 2] == ')'):
        element_2 = element_2.replace(' ', ')', 1)
      element_2 = element_2.replace(' ', '')

      element_3 = formula_in_process[position - 3]
      element_3 = element_3.replace('(', '', 1)

      formula_in_process[position - 1 ] = "(" + element_3 + " V " + element_1 + ")"
      formula_in_process[position] = '∧'
      formula_in_process[position + 1] = "(" + element_2 + " V " + element_1 + ")"
      del formula_in_process[position - 2]
      del formula_in_process[position - 3]
      have_transformation = True

  formula_final = ' '.join(formula_in_process)
  return formula_final, have_transformation

def convert(type_of_convertion, formula):
  if(type_of_convertion == "conjuntiva"):
    formula_begin = formula
    have_transformation = True
    while(have_transformation == True):
      # Vai transformando até a formula não ter mais transformação
      formula_begin, have_transformation = transform_conj(formula_begin)
    return formula_begin
  elif(type_of_convertion == "disjuntiva"):
    formula_begin = formula
    have_transformation = True
    while(have_transformation == True):
      # Vai transformando até a formula não ter mais transformação
      formula_begin, have_transformation = transform_dis(formula_begin)
    return formula_begin
  else:
    return "--- Tipo de transformação invalida!!! ---"

In [ ]:
fp = open("Filebase_v1.txt", 'r')
lines = fp.readlines()
for line in lines:
  print("----------------------------------------")
  print("Formula de entrada: ")
  display(Latex(f'${convert_to_latex(line)}$'))
  print("Formula em FNC: ")
  display(Latex(f'${convert_to_latex(convert("conjuntiva", line))}$'))
  print("Formula em FND: ")
  display(Latex(f'${convert_to_latex(convert("disjuntiva", line))}$'))
  print("----------------------------------------")

----------------------------------------
Formula de entrada: 


<IPython.core.display.Latex object>

Formula em FNC: 


<IPython.core.display.Latex object>

Formula em FND: 


...

Formula em FNC: 


<IPython.core.display.Latex object>

Formula em FND: 


<IPython.core.display.Latex object>

----------------------------------------
